In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
import sqlite3 as sq
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
matplotlib.use('TkAgg')
from tkinter import filedialog
 

In [2]:
# parse data using built-in read_csv function in pandas 
airports=pd.read_csv('airports.csv')
airfrequency=pd.read_csv('airport-frequencies.csv')
runways=pd.read_csv('runways.csv')

In [3]:
# clean frequency dataset
airfrequency=airfrequency.drop_duplicates()

airfrequency['airport_ident']=airfrequency['airport_ident'].astype(str)
airfrequency['airport_ident']=airfrequency['airport_ident'].str.upper()

airfrequency['type']=airfrequency['type'].str.upper()
airfrequency['type'].replace({"SAFETY COMM":"SAFETYCOM","SAFETY COM":"SAFETYCOM"}, inplace = True)

airfrequency.loc[airfrequency['id']==298892,'frequency_mhz']=131.7
airfrequency.loc[airfrequency['id']==308254,'frequency_mhz']=123.5
airfrequency.loc[airfrequency['id']==313875,'frequency_mhz']=122.8

airfrequency=airfrequency.loc[~(airfrequency['frequency_mhz']==0)]

airfrequency.loc[airfrequency['frequency_mhz']==0]

,id,airport_ref,airport_ident,type,description,frequency_mhz


In [4]:
# clean runway dataset
runways=runways.drop_duplicates()  

runways['airport_ident']=runways['airport_ident'].astype(str)
ident_of_airport=list(airports.loc[~(airports['type']=='closed'),'ident'])
name_of_airport=list(airports.loc[~(airports['type']=='closed'),'name'])
dict_runway=dict(zip(ident_of_airport,name_of_airport))
runways['airport_name'] = runways.apply(lambda x: dict_runway[x['airport_ident']] if x['airport_ident'] 
                                        in dict_runway else 'not known', axis=1)
runways_for_airport_not_closed=runways.loc[~(runways['airport_name']=='not known')]
runways_for_airport_not_closed.head()

,id,airport_ref,airport_ident,length_ft,width_ft,surface,lighted,closed,le_ident,le_latitude_deg,...,le_elevation_ft,le_heading_degT,le_displaced_threshold_ft,he_ident,he_latitude_deg,he_longitude_deg,he_elevation_ft,he_heading_degT,he_displaced_threshold_ft,airport_name
0,269408,6523,00A,80.0,80.0,ASPH-G,1,0,H1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total Rf Heliport
1,255155,6524,00AK,2500.0,70.0,GRVL,0,0,N,NaN,...,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,Lowell Field
2,254165,6525,00AL,2300.0,200.0,TURF,0,0,01,NaN,...,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,Epps Airpark
4,322128,322127,00AS,1450.0,60.0,Turf,0,0,1,NaN,...,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,Fulton Airport
5,257681,6527,00AZ,1700.0,60.0,GRAVEL,0,0,15,NaN,...,NaN,NaN,NaN,33,NaN,NaN,NaN,NaN,NaN,Cordes Airport


In [5]:
#clean airports dataset 
airports=airports.drop_duplicates()
airports['ident']=airports['ident'].astype(str)

# filling missing values in airports.csv
# for some rows, where the values in columns 'gps_code' and 'local_code' are both missing, they are saved in 'keywords' instead and are the same as'ident' 
filt=airports['gps_code'].isnull() & airports['local_code'].isnull() & airports['ident'].isin(airports['keywords'])
val = airports.loc[filt,'keywords']
airports.loc[filt,['local_code','gps_code']] =val
airports.loc[airports['ident']==airports['keywords'],'keywords']= np.nan

# filtering out closed airports 
airports=airports.loc[~(airports['type']=='closed')]
airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
5,322127,00AS,small_airport,Fulton Airport,34.942803,-97.818019,1100.0,NaN,US,US-OK,Alex,no,00AS,NaN,00AS,NaN,NaN,NaN


In [6]:
# save cleaned dataset and create database
connection1 =sq.connect('airportsdb.db')
curs1=connection1.cursor()
airports.to_sql('airportsInfo',connection1, if_exists='replace',index=False)
connection1.close()

connection2 =sq.connect('airport-frequencies.db')
curs2=connection2.cursor()
airfrequency.to_sql('airportsFrequencyInfo',connection2, if_exists='replace',index=False)
connection2.close()

connection3 =sq.connect('runwaysdb.db')
curs3=connection3.cursor()
runways.to_sql('airportRunwaysInfo',connection3, if_exists='replace',index=False)
connection3.close()

In [7]:
# fetch cleaned data in data base for further data manipulation in python
connection1 =sq.connect('airportsdb.db')
airports=pd.read_sql_query("SELECT * from airportsInfo",connection1) 
connection1.close()

connection2 =sq.connect('airport-frequencies.db')
airfrequency=pd.read_sql_query("SELECT * from airportsFrequencyInfo",connection2) 
connection2.close()

connection3 =sq.connect('runwaysdb.db')
runways=pd.read_sql_query("SELECT * from airportRunwaysInfo",connection3) 
connection3.close()

airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,None,US,US-PA,Bensalem,no,00A,None,00A,None,None,None
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,None,US,US-KS,Leoti,no,00AA,None,00AA,None,None,None
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,None,US,US-AK,Anchor Point,no,00AK,None,00AK,None,None,None
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,None,US,US-AL,Harvest,no,00AL,None,00AL,None,None,None
4,322127,00AS,small_airport,Fulton Airport,34.942803,-97.818019,1100.0,None,US,US-OK,Alex,no,00AS,None,00AS,None,None,None


In [8]:
# reshape data for outputs meeting clients' requirements 
ident_of_airport=list(airports.loc[~(airports['type']=='closed'),'ident'])
name_of_airport=list(airports.loc[~(airports['type']=='closed'),'name'])
dict_runway=dict(zip(ident_of_airport,name_of_airport))
airfrequency['airport_name'] = airfrequency.apply(lambda x: dict_runway[x['airport_ident']] 
                                                  if x['airport_ident'] in dict_runway else 'not known', axis=1)
airfrequency=airfrequency.loc[~(airfrequency['airport_name']=='not known')]

# filtering out non-UK airports and keeping only small, medium and large airports

uk_Airports_by_type=airports.loc[~(airports['type']=='closed')&(airports['iso_country']=='GB') &
                                 ((airports['type']=='small_airport')|(airports['type']=='medium_airport')|
                                   (airports['type']=='large_airport'))]

# create a dictionary for airport_ident and airport_type
ident=list(uk_Airports_by_type['ident'])
air_type=list(uk_Airports_by_type['type'])
airport_type=dict(zip(ident,air_type))

# using the airport_ident in the dictionary created to match frequency in the data
# so frequencies can be matched to selected UK airports 

airfrequency['airport_type'] = airfrequency.apply(lambda x: airport_type[x['airport_ident']] 
                                                  if x['airport_ident'] in airport_type else 'not known', axis=1)

# reshape data - filerting out irrelevant data.
# I have kept airport_ref so as to identify airports using reference number rather than ident if needed, and i have also kept 'type'to identify the type of frequency if needed
airfrequency_UK_airports=airfrequency.loc[:,['airport_ref','airport_ident', 
                                             'airport_name','type','description','frequency_mhz', 'airport_type']]

# reshape data - adding 3 columns,small_airport,medium_airport and large_airport
airfrequency_UK_airports.loc[airfrequency_UK_airports['airport_type']=='small_airport','small_airport']='Yes'
airfrequency_UK_airports.loc[airfrequency_UK_airports['airport_type']=='medium_airport','medium_airport']='Yes'
airfrequency_UK_airports.loc[airfrequency_UK_airports['airport_type']=='large_airport','large_airport']='Yes'

airfrequency_UK_airports=airfrequency_UK_airports.loc[(airfrequency_UK_airports['airport_type']=='small_airport')|
                                                      (airfrequency_UK_airports['airport_type']=='large_airport')|
                                                     (airfrequency_UK_airports['airport_type']=='medium_airport')]
# del airfrequency_UK_airports['airport_type']

airfrequency_UK_airports.head()

,airport_ref,airport_ident,airport_name,type,description,frequency_mhz,airport_type,small_airport,medium_airport,large_airport
2774,307538,EG03,Badminton Airfield,A/G,Badminton Radio,123.175,small_airport,Yes,NaN,NaN
2775,28473,EG64,Rufforth West,INFO,None,129.975,small_airport,Yes,NaN,NaN
2776,3075,EG74,Bruntingthorpe Airport,A/G,RDO,122.825,small_airport,Yes,NaN,NaN
2777,2385,EGAA,Belfast International Airport,APP,APP,128.500,large_airport,NaN,NaN,Yes
2778,2385,EGAA,Belfast International Airport,ATIS,ATIS,128.200,large_airport,NaN,NaN,Yes


In [9]:
# save a preapred reshaped dataset into database

conn=sq.connect('airfrequency.db')
airfrequency_UK_airports.to_sql('uk_airportsFrequencyInfo',conn, if_exists='replace',index=False)
curs=conn.cursor()
conn.close()

In [10]:
# checking frequently used frequencies to see if there is any corelationshiop between different size airports.
a=airfrequency.groupby(['frequency_mhz','airport_type']).size().unstack()
a.dropna()

airport_type,large_airport,medium_airport,not known,small_airport
frequency_mhz,,,,
122.100,3.0,22.0,489.0,2.0
132.475,1.0,1.0,5.0,1.0


In [11]:
# GUI functions to show results of mean, mode, median required by client
def mean_of_large_UK_airport():
    
    conn=sq.connect('airfrequency.db')
    df=pd.read_sql_query("SELECT * from uk_airportsFrequencyInfo",conn)       
    conn.close() 
    return df.loc[df['large_airport']=='Yes','frequency_mhz'].mean()

def median_of_large_UK_airport():
    
    conn=sq.connect('airfrequency.db')
    df=pd.read_sql_query("SELECT * from uk_airportsFrequencyInfo",conn)       
    conn.close() 
    return df.loc[df['large_airport']=='Yes','frequency_mhz'].median()

def mode_of_large_UK_airport():
    
    conn=sq.connect('airfrequency.db')
    df=pd.read_sql_query("SELECT * from uk_airportsFrequencyInfo",conn)       
    conn.close() 
    return df.loc[df['large_airport']=='Yes','frequency_mhz'].mode()

mean_of_large_UK_airport()

120.00752293577983

In [12]:
median_of_large_UK_airport()

121.85

In [13]:
mode_of_large_UK_airport()

0    121.75
1    122.10
Name: frequency_mhz, dtype: float64

In [14]:
# GUI functions to show results of mean, mode, median required by client

def mean_for_frequency_over_100_UK_airports():

    conn=sq.connect('airfrequency.db')
    df=pd.read_sql_query("SELECT * from uk_airportsFrequencyInfo",conn)       
    conn.close()                  
    return df.loc[df['frequency_mhz']>100 ,'frequency_mhz'].mean()

def median_for_frequency_over_100_UK_airports():

    conn=sq.connect('airfrequency.db')
    df=pd.read_sql_query("SELECT * from uk_airportsFrequencyInfo",conn)       
    conn.close()               
    return df.loc[df['frequency_mhz']>100 ,'frequency_mhz'].median()

def mode_for_frequency_over_100_UK_airports():

    conn=sq.connect('airfrequency.db')
    df=pd.read_sql_query("SELECT * from uk_airportsFrequencyInfo",conn)       
    conn.close()      
    return airfrequency_UK_airports.loc[airfrequency_UK_airports['frequency_mhz']>100 ,'frequency_mhz'].mode()

In [15]:
mode_for_frequency_over_100_UK_airports()

0    122.1
Name: frequency_mhz, dtype: float64

In [16]:
mean_for_frequency_over_100_UK_airports()

149.27128571428574

In [17]:
median_for_frequency_over_100_UK_airports()

124.4

In [18]:
# GUI functions to show frquencies used by client - to be triggered by events

def show_frequency_for_small_airports():
    
    conn=sq.connect('airfrequency.db')
    df=pd.read_sql_query("SELECT * from uk_airportsFrequencyInfo",conn)       
    conn.close()  
   
    df['type']=df['type'].str.upper()
    df['type'].replace({"SAFETY COMM":"SAFETYCOM","SAFETY COM":"SAFETYCOM"}, inplace = True)
   
    small_airport_freq = df.loc[df['airport_type']=="small_airport"]
    
    small_airport_groupby = small_airport_freq.groupby('type')
   
    a=small_airport_groupby['frequency_mhz'].agg('mean')
    a=a.reset_index()
    
    x=a['type']
    y=a['frequency_mhz']
    plt.plot(x,y,'ko--')
    plt.xticks(x, rotation='vertical',size=8)
    plt.ylabel('frequency')
    plt.xlabel('small_airport')
    plt.show()    

In [19]:
# GUI functions to generate graphics to demonstrate relationships of airports in different sizes by anaylising frequencies used

def show_frequency_trends():

    conn=sq.connect('airfrequency.db')
    airfrequency=pd.read_sql_query("SELECT * from uk_airportsFrequencyInfo",conn)       
    conn.close()  
    
    airfrequency['type']=airfrequency['type'].str.upper()
    airfrequency['type'].replace({"SAFETY COMM":"SAFETYCOM","SAFETY COM":"SAFETYCOM"}, inplace = True)

    small_airport_freq = airfrequency.loc[airfrequency['airport_type']=="small_airport"]
    medium_airport_freq = airfrequency.loc[airfrequency['airport_type']=="medium_airport"]
    large_airport_freq = airfrequency.loc[airfrequency['airport_type']=="large_airport"]

    x1=list(small_airport_freq['frequency_mhz'])
    x2=list(medium_airport_freq['frequency_mhz'])
    x3=list(large_airport_freq['frequency_mhz'])

    colors = ['#E69F00', '#56B4E9', '#F0E442']
    names = ['small airport', 'medium airport', 'large airport']

    plt.hist([x1, x2, x3], bins = 20, density=True,
             color = colors, label=names)

    _=plt.xlabel('frequency')
    _=plt.ylabel('density')
    
    plt.xlim([20,400])
 
    plt.tight_layout()
    plt.legend()
    plt.show()
    
    bins = np.array([10,50,100,110,120,130,140,150,200,250,300,500,750])
    labels=pd.cut(airfrequency['frequency_mhz'],bins)
    grouped=airfrequency.groupby(['airport_type',labels])
    newdf=grouped.size().unstack(0)
    newdf.plot(kind='barh',figsize=(8,6,),ylabel='times',rot=15)
    
    fig,axes=plt.subplots(2,2,figsize=(10,10,))
    sns.boxplot(ax=axes[0,0],x='frequency_mhz',y='airport_type',data=airfrequency[airfrequency['frequency_mhz']<100])
    sns.boxplot(ax=axes[0,1],x='frequency_mhz',y='airport_type',data=airfrequency[(airfrequency['frequency_mhz']<140)&(airfrequency['frequency_mhz']>120)])                                                                              
    sns.boxplot(ax=axes[1,0],x='frequency_mhz',y='airport_type',data=airfrequency[(airfrequency['frequency_mhz']<120)&(airfrequency['frequency_mhz']>100)])                                                                             
    sns.boxplot(ax=axes[1,1],x='frequency_mhz',y='airport_type',data=airfrequency[airfrequency['frequency_mhz']>140])

In [20]:
# GUI functions for the second window 

def select_record(e):
    
    global rowid_entry  
    global airport_ref_entry
    global airport_ident_entry
    global airport_name_entry
    global type_entry
    global description_entry
    global frequency_mhz_entry
    global airport_type_entry
    
    #     CLEAR ENTRY BOXES
    rowid_entry.delete(0,END)
    airport_ref_entry.delete(0,END)
    airport_ident_entry.delete(0,END)
    airport_name_entry.delete(0,END)
    type_entry.delete(0,END)
    description_entry.delete(0,END)
    frequency_mhz_entry.delete(0,END)
    airport_type_entry.delete(0,END)
   
    
    #      GRAB RECORD NUMBER
    global my_tree
    selected=my_tree.focus()
    values=my_tree.item(selected,'values')
    
    #   outputs to entry boxes
    rowid_entry.insert(0,values[0])
    airport_ref_entry.insert(0,values[1])
    airport_ident_entry.insert(0,values[2])
    airport_name_entry.insert(0,values[3])
    type_entry.insert(0,values[4])
    description_entry.insert(0,values[5])
    frequency_mhz_entry.insert(0,values[6])
    airport_type_entry.insert(0,values[7])

def clear_entry():
    #     CLEAR ENTRY BOXES
    global rowid_entry  
    global airport_ref_entry
    global airport_ident_entry
    global airport_name_entry
    global type_entry
    global description_entry
    global frequency_mhz_entry
    global airport_type_entry
    
    
    rowid_entry.delete(0,END)
    airport_ref_entry.delete(0,END)
    airport_ident_entry.delete(0,END)
    airport_name_entry.delete(0,END)
    type_entry.delete(0,END)
    description_entry.delete(0,END)
    frequency_mhz_entry.delete(0,END)
    airport_type_entry.delete(0,END)

#    Save update button event
def update_record():
    
    try:
        frequency_updated=float(frequency_mhz_entry.get())
        
        conn=sq.connect('airfrequency.db')
        c=conn.cursor()
        c.execute("""UPDATE uk_airportsFrequencyInfo SET
              frequency_mhz = :frequency_mhz,
              description=:description,
              type =:type
        
            WHERE oid=:oid""",
            {                 
              'frequency_mhz': frequency_updated,
                'type':type_entry.get().upper(),
               'description':description_entry.get(),
               'oid': rowid_entry.get()
              
            })
    
        conn.commit()
        conn.close()
        
        messagebox.showinfo('', 'Updated Successfully')
    
    except ValueError: 
        messagebox.showerror('Invalid Entry', 'Type a valid frequency')

            
def clear_all():
    global my_tree
    for item in my_tree.get_children():
        my_tree.delete(item)

    
#create a query databse function 
def load_database():
    connection1 =sq.connect('airfrequency.db')
    curs1=connection1.cursor()
    curs1.execute("SELECT rowid,*FROM uk_airportsFrequencyInfo")
    records=curs1.fetchall()
    global count 
    count=0
    for record in records:
        if count%2 ==0:
            my_tree.insert(parent='', index= 'end', iid=count, text='', 
                           values=(record[0],record[1],record[2],record[3],record[4],
                           record[5],record[6],record[7],
                           record[8],record[9],record[10]),tags=('evenrow',))
        else:
            my_tree.insert(parent='', index= 'end', iid=count, text='', 
                           values=(record[0],record[1],record[2],record[3],record[4],
                           record[5],record[6],record[7],
                           record[8],record[9],record[10]),tags=('oddrow',))
        count = count+1
    connection1.commit()
    connection1.close()
    
def clicked_1():
    global r1
    value = r1.get()
    if value== 1:
        text_in=mean_of_large_UK_airport()
        
    elif value==2:
        text_in=median_of_large_UK_airport()
        
    else: 
        text_in=mode_of_large_UK_airport()
    
    myLabel_1.config(text=text_in)

        
def clicked_2():
    global r2 
    value = r2.get()
    if value== 1:
        text_in=mean_for_frequency_over_100_UK_airports()
        
    elif value==2:
        text_in=median_for_frequency_over_100_UK_airports()
        
    else: 
        text_in=mode_for_frequency_over_100_UK_airports()
    
    myLabel_2.config(text=text_in)


In [21]:
# this is the GUI design for second window to show outputs required by client and data analysis 

def open_a_new_window():
    
    top = Toplevel()
    top.title('UK Airport Data Analysis')
    
    #     btn2=Button(top, text="close window", command=top.destroy).pack()

    style = ttk.Style()
    style.theme_use('default')

    #     created the treeview colour

    style.configure("Treeview",
                    background= "#D3D3D3",
                    foreground="black",
                    rowheight=25,
                    fieldbackground="#D3D3D3")

    style.map('Treeview',
              background=[('selected',"#347083")])

    tree_frame= Frame(top)
    tree_frame.pack(fill=BOTH, expand=1, pady=10)

    tree_scroll=Scrollbar(tree_frame)
    tree_scroll.pack(side=RIGHT,fill=Y)

    global my_tree
    my_tree=ttk.Treeview(tree_frame,yscrollcommand=tree_scroll.set,selectmode="extended")
    my_tree.pack()

    tree_scroll.config(command=my_tree.yview)

    my_tree['columns'] =('Rowid','airport_ref','airport_ident','airport_name',
                     'type','description','frequency_mhz',
                     'airport_type','small_airport','medium_airport','large_airport')

    my_tree.column("#0", width=0, stretch=0)
    my_tree.heading("#0", text="",anchor=CENTER)

    for item in my_tree['columns']:
        my_tree.column(item, anchor=W, width=80)
        my_tree.heading(item, text=item,anchor=CENTER) 

    my_tree.tag_configure('oddrow', background="white")
    my_tree.tag_configure('evenrow', background="lightgreen")

    global data_frame
    data_frame=LabelFrame(top, text="Airport Info")
    data_frame.pack(fill="x",expand="no", padx=5,pady=5)

    global rowid_label
    rowid_label=Label(data_frame,text="Row ID")
    rowid_label.grid(row=0,column=0, padx=5, pady=5)
    global rowid_entry
    rowid_entry = Entry(data_frame)
    rowid_entry.grid(row=0,column=1, padx=5,pady=5)

    global airport_ref_label
    airport_ref_label=Label(data_frame,text="Airport Reference")
    airport_ref_label.grid(row=0,column=2, padx=5, pady=5)
    global airport_ref_entry
    airport_ref_entry = Entry(data_frame)
    airport_ref_entry.grid(row=0,column=3, padx=5,pady=5)

    global airport_ident_label
    airport_ident_label=Label(data_frame,text="Airport ident")
    airport_ident_label.grid(row=1,column=0, padx=5, pady=5)
    global airport_ident_entry
    airport_ident_entry = Entry(data_frame)
    airport_ident_entry.grid(row=1,column=1, padx=5,pady=5)

    global airport_name_label
    airport_name_label=Label(data_frame,text="Airport name")
    airport_name_label.grid(row=1,column=2, padx=5, pady=5)
    global airport_name_entry 
    airport_name_entry = Entry(data_frame)
    airport_name_entry.grid(row=1,column=3, padx=5,pady=5)

    global airport_type_label
    airport_type_label=Label(data_frame,text="Airport Type")
    airport_type_label.grid(row=1,column=4, padx=5, pady=5)
    global airport_type_entry
    airport_type_entry = Entry(data_frame)
    airport_type_entry.grid(row=1,column=5, padx=5,pady=5)

    global button_frame
    button_frame = LabelFrame(top, text='Frequency Details')
    button_frame.pack(fill="x",expand="no",padx=5,pady=5)

    global type_label
    type_label=Label(button_frame,text="Frequency Type")
    type_label.grid(row=0,column=0, padx=5, pady=5)
    global type_entry
    type_entry = Entry(button_frame,)
    type_entry.grid(row=0,column=1, padx=5,pady=5)

    global description_label
    description_label=Label(button_frame,text="Frequency Description")
    description_label.grid(row=0,column=2, padx=5, pady=5)
    global description_entry
    description_entry = Entry(button_frame,)
    description_entry.grid(row=0,column=3, padx=5,pady=5)

    global frequency_mhz_label
    frequency_mhz_label=Label(button_frame,text="frequency_mhz")
    frequency_mhz_label.grid(row=0,column=4, padx=5, pady=5)
    global frequency_mhz_entry
    frequency_mhz_entry = Entry(button_frame,)
    frequency_mhz_entry.grid(row=0,column=5, padx=5,pady=5)

    global update_button
    update_button=Button(button_frame, text="Save Update", command=update_record)
    update_button.grid(row=1,column=0, padx=5,pady=5)

    global select_button
    select_button=Button(button_frame, text="Clear Entry", command=clear_entry)
    select_button.grid(row=1,column=1, padx=5,pady=5)

    my_tree.bind("<ButtonRelease-1>",select_record)

    global radiobutton_frame
    radiobutton_frame = LabelFrame(top, text='Frequency analysis')
    radiobutton_frame.pack(fill="x",expand="no", padx=5,pady=5)

    global large_airport_frame
    large_airport_frame= LabelFrame(radiobutton_frame, text='large airports',width=350, height=250)
    large_airport_frame.grid(row=0,column=0,padx=5,pady=5)


    global r1
    r1 = IntVar()
    r1.get()

    Radiobutton(large_airport_frame, text="mean", variable=r1,value=1,command=clicked_1).grid(row=0,column=0,
                                                                                              padx=5,pady=5)
    Radiobutton(large_airport_frame, text="median", variable=r1,value=2,command=clicked_1).grid(row=1,column=0,
                                                                                                padx=5,pady=5)
    Radiobutton(large_airport_frame, text="mode",variable=r1, value=3,command=clicked_1).grid(row=2,column=0,
                                                                                              padx=5,pady=5)

    global myLabel_1
    myLabel_1=Label(large_airport_frame,text="")
    myLabel_1.grid(row=3,column=0,padx=5,pady=5)


    global frequencyv_over100_frame
    
    frequencyv_over100_frame= LabelFrame(radiobutton_frame, text='airports with frequency over 100',width=350, 
                                         height=250)
    frequencyv_over100_frame.grid(row=0,column=4,padx=5,pady=5)
 
    global r2
    r2 = IntVar()
    r2.get()

    Radiobutton(frequencyv_over100_frame, text="mean", variable=r2,value=1,command=clicked_2).grid(row=0,padx=5,pady=5)
    Radiobutton(frequencyv_over100_frame, text="median",variable=r2,value=2,command=clicked_2).grid(row=1,padx=5,pady=5)
    Radiobutton(frequencyv_over100_frame, text="mode",variable=r2, value=3,command=clicked_2).grid(row=2,padx=5,pady=5)
 
    global myLabel_2
    myLabel_2=Label(frequencyv_over100_frame,text="")
    myLabel_2.grid(row=3,padx=5,pady=5)


    global graph_frame
    graph_frame= LabelFrame(radiobutton_frame, text='Graph',width=350, height=250)
    graph_frame.grid(row=0,column=8,padx=5,pady=5)

    global button_graph_1
    button_graph_1=Button(graph_frame,text="show frequencies used by small airports", 
                      command=show_frequency_for_small_airports)
    button_graph_1.grid(row=0,padx=10,pady=5)

    global button_graph_2
    button_graph_2=Button(graph_frame,text="show frequencies trend by size of airports", 
                          command=show_frequency_trends)
    button_graph_2.grid(row=1,padx=10,pady=5)

    load_database()


In [22]:
# button functions for the main window

def File_dialog():
    filename=filedialog.askopenfilename(initialdir='/',title='select a file',filetypes=(("cxv files", "*.csv"),("all files", "*.*")))
    label_file['text'] = filename

    
def load_airport():
    
#     clear existing data in Treeview if any
    clear_all()
    
#     parse airports csv
    airports1=pd.read_csv('airports.csv')
        
#   load data and disply it in Treeview 

    tv1["column"] =list(airports1.columns)
    tv1["show"] ="headings"
    
    for column in tv1['columns']:
        tv1.heading(column,text=column)
        
    airports1_rows=airports1.to_numpy().tolist()
    
    for row in airports1_rows:
        tv1.insert("","end", values=row)


def load_frequency():
    
    frequency1=pd.read_csv('airport-frequencies.csv')  
    
    clear_all()
    
    tv1["column"] =list(frequency1.columns)
    tv1["show"] ="headings"
    for column in tv1['columns']:
        tv1.heading(column,text=column)
        
    frequency1_rows=frequency1.to_numpy().tolist()
    
    for row in frequency1_rows:
        tv1.insert("","end", values=row)

def load_runways():
    
    clear_all()
    
    runways1=pd.read_csv('runways.csv') 
    
    tv1["column"] =list(runways1.columns)
    tv1["show"] ="headings"
    for column in tv1['columns']:
        tv1.heading(column,text=column)
        
    runways1_rows=runways1.to_numpy().tolist()
    
    for row in runways1_rows:
        tv1.insert("","end", values=row)

def load_cleaned_airportdata():
    
    clear_all()
    
    connection1 =sq.connect('airportsdb.db')
    airports1=pd.read_sql_query("SELECT * from airportsInfo",connection1) 
    connection1.close()
    
    tv1["column"] =list(airports1.columns)
    tv1["show"] ="headings"
    
    for column in tv1['columns']:
        tv1.heading(column,text=column)
        
    airports1_rows=airports1.to_numpy().tolist()
    
    for row in airports1_rows:
        tv1.insert("","end", values=row)


def load_cleaned_frequencydata():
    
    connection2 =sq.connect('airport-frequencies.db')
    airfrequency1=pd.read_sql_query("SELECT * from airportsFrequencyInfo",connection2) 
    connection2.close()
    
    clear_all()
    
    tv1["column"] =list(airfrequency1.columns)
    tv1["show"] ="headings"
    for column in tv1['columns']:
        tv1.heading(column,text=column)
    frequency1_rows=airfrequency1.to_numpy().tolist()
    
    for row in frequency1_rows:
        tv1.insert("","end", values=row)

def load_cleaned_runwaysdata():
     
    clear_all()
    
    connection3 =sq.connect('runwaysdb.db')
    runways=pd.read_sql_query("SELECT * from airportRunwaysInfo",connection3) 
    connection3.close()
    
    tv1["column"] =list(runways.columns)
    tv1["show"] ="headings"
    for column in tv1['columns']:
        tv1.heading(column,text=column)
        
    runways1_rows=runways.to_numpy().tolist()
    
    for row in runways1_rows:
        tv1.insert("","end", values=row)
    
    
def clear_all():
    for item in tv1.get_children():
        tv1.delete(item)

In [23]:
root = Tk()
root.title('General Airport Data')
root.geometry("1000x500")

frame1 = LabelFrame(root, text="View data")
frame1.pack(fill="x",expand="no", padx=5,pady=5)

file_frame =LabelFrame(root,text="View Initial Data Set")
file_frame.pack(fill="x",expand="no", padx=5,pady=5)

button1 =Button(file_frame, text="Airport Info", command=load_airport)
button1.grid(row=0,column=0,padx=5,pady=5)

button2=Button(file_frame, text="Frequency Data", command =load_frequency)
button2.grid(row=0,column=1,padx=5,pady=5)

button3=Button(file_frame, text="Runway Data", command = load_runways)
button3.grid(row=0,column=2,padx=5,pady=5)

cleanfile_frame =LabelFrame(root,text="Clean Initial Data Set")
cleanfile_frame.pack(fill="x",expand="no", padx=5,pady=5)

button4 =Button(cleanfile_frame, text="Airport Info", command=load_cleaned_airportdata)
button4.grid(row=0,column=0,padx=5,pady=5)

button5=Button(cleanfile_frame, text="Frequency Data", command =load_cleaned_frequencydata)
button5.grid(row=0,column=1,padx=5,pady=5)

button6=Button(cleanfile_frame, text="Runway Data", command = load_cleaned_runwaysdata)
button6.grid(row=0,column=2,padx=5,pady=5)

option_frame =LabelFrame(root,text="Options")
option_frame.pack(fill="x",expand="no", padx=5,pady=5)

button7=Button(option_frame, text="UK airport analysis", command = open_a_new_window)
button7.grid(row=0,column=0,padx=5,pady=5)

button9=Button(option_frame, text="Clear", command = clear_all)
button9.grid(row=0,column=2,padx=5,pady=5)


# label_file=Label(file_frame, text="no file selected")
# label_file.grid(row=0,column=0,padx=5,pady=5)

tv1=ttk.Treeview(frame1)
tv1.pack(fill=BOTH,expand="yes", padx=5,pady=5)

treescrolly=Scrollbar(frame1,orient='vertical',command=tv1.yview)
treescrollx=Scrollbar(frame1,orient='horizontal',command=tv1.xview)

tv1.config(xscrollcommand=treescrollx.set,yscrollcommand=treescrolly.set)
treescrollx.pack(side='bottom',fill='x')
treescrolly.pack(side='right',fill='y')



root.mainloop()